In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import string
import pickle
import re

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import lda
from sklearn.feature_extraction.text import CountVectorizer

In [175]:
# load the data
df1 = pd.read_csv("data/Atlanta, GA.csv")
df2 = pd.read_csv("data/Austin, TX.csv")
df3 = pd.read_csv("data/Boston, MA.csv")
df4 = pd.read_csv("data/Cambridge, MA.csv")
df5 = pd.read_csv("data/Chicago, IL.csv")
df6 = pd.read_csv("data/Los Angeles, CA.csv")
df7 = pd.read_csv("data/New York, NY.csv")
df8 = pd.read_csv("data/Palo Alto, CA.csv")
df9 = pd.read_csv("data/Philadelphia, PA.csv")
df10 = pd.read_csv("data/San Diego, CA.csv")
df11 = pd.read_csv("data/San Jose, CA.csv")
df12 = pd.read_csv("data/San Fransisco, CA.csv")
df13 = pd.read_csv("data/Seattle, WA.csv")
df14 = pd.read_csv("data/Washington, DC.csv")

# combine data
frame = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14]
data_df = pd.concat(frame, axis = 0, ignore_index = True)

In [210]:
data_df = data_df.dropna(subset=['job_description']).reset_index(drop=True)

In [3]:
# convert min_salary and max_salary columns to int
data_df['Min_Salary'] = data_df['Min_Salary'].apply(lambda x : int(x))
data_df['Max_Salary'] =data_df['Max_Salary'].apply(lambda x : int(x))

In [4]:
# Extracting date and day from Date_Posted : data is the format y-m-d
import calendar
data_df['Month'] = data_df['Date_Posted'].apply(lambda x : calendar.month_abbr[int(str(x).split('-')[1])]) 

In [5]:
def Convert_to_Day(x):
    sl = x.split('-')
    
    return calendar.day_abbr[int(calendar.weekday(int(sl[0]) , int(sl[1]) , int(sl[2])))]

data_df['Day'] = data_df['Date_Posted'].apply(lambda x : Convert_to_Day(x))

In [6]:
# While collecting the data if no salary is found I replaced the value by -1 so lets store that data in different data frame
index_missing = data_df[(data_df['Min_Salary'] == -1)].index
test_df = data_df.iloc[index_missing, :].reset_index(drop = True)
data_df.drop(index_missing , axis = 0 , inplace = True)
data_df = data_df.reset_index(drop = True)

In [7]:
#Now that we have train and test set there are duplicates in the data becasue our scraper was not perfect and could havea assimilated multiple entries
cols = [col for col in data_df.columns if col not in ['Day' , 'Month']]
#For training data 
train_series = data_df.duplicated(cols , keep = 'first')
data_df =data_df[~train_series].reset_index(drop = True)
test_series = test_df.duplicated(cols , keep = 'first')
test_df = test_df[~test_series].reset_index(drop = True)


In [121]:
data_df.to_csv('train.csv')

# Preprocessing Data

In [211]:
# Combine the desciptions by the job tilte
data = data_df[['job_description']]
data

,job_description
0,SENIOR DATA SCIENTIST- Temporary Assignment (R...
1,Data Scientist Location Atlanta - Midtown Dep...
2,Sharecare is the digital health company that h...
3,Short term work for 2 mid’s or 1 sr is needed ...
4,Business Intelligence Analyst Location Atlant...
...,...
5499,"Working out of our Chevy Chase, MD/Washington ..."
5500,Our purpose is to serve the nation with the si...
5501,Trewon is a competitive firm providing enterpr...
5502,The Data Consumption team of Decision Sciences...


In [212]:
# Create a function to clean text data
def clean_text(text):
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text).lower() #remove punctutations
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n',' ',text)
    return text

In [213]:
# Clean the text data and remove the job title 'Others'
clean = lambda x :clean_text(x)
df_clean = pd.DataFrame(data_df['job_description'].apply(clean))
df_clean = df_clean[df_clean.index != 'Others'].copy()

/Users/nguyengiang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:122: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = op(self.values, np.asarray(other))


In [214]:
# Lemmentize the text data to improve analysis
lemmer = WordNetLemmatizer()
df_clean['job_description'] = df_clean['job_description'].apply(lambda x: word_tokenize(x))
df_clean['job_description'] = df_clean['job_description'].apply(lambda x : [lemmer.lemmatize(y) for y in x])
df_clean['job_description'] = df_clean['job_description'].apply(lambda x: " ".join(x))

In [215]:
# Add words that frequently appear in the descriptions but carry no value to the list of stop words
from sklearn.feature_extraction import text
extra_stopword = ['data','experience','work','team','will','skill','year','skills']
stop_words = text.ENGLISH_STOP_WORDS.union(extra_stopword)

In [216]:
text = data_df['job_description'].values

## Year of experience

In [217]:
data_df['experience_req'] = np.nan
for t in range(len(text)):
    for sentence in text[t].split('\n'):
        print(sentence)
        if 'years' in sentence:
            year = re.findall("\d{1,2}\+? years", sentence)
            print(year)
            if len(year)==1:
                num = year[0].split(' ')
                data_df['experience_req'][t] = num[0]
        
        
        

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

<ipython-input-217-ec1b65e692e1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['experience_req'][t] = num[0]
/Users/nguyengiang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value


The Organizational Research and Measurement team conducts research supporting all Amazon corporate employees. Our goal is to build talent systems to enable employees to thrive at Amazon. We focus on the entire employee life-cycle to improve both business and employee outcomes. This entails doing longitudinal survey research, organizational network analysis, experimental and quasi-experimental studies for causal inference, building services that plug-in to other tools, and providing general consultation to stakeholders. We aim to improve the outcomes of our business and employees by doing cutting edge social science research.The Role:In this Data Scientist role, you will do causal data science, build predictive models, conduct simulations, create visualizations, and influence data science practice.Amazon is an Equal Opportunity Employer Minority / Female / Disability / Veteran / Gender Identity / Sexual Orientation.Basic Qualifications· Proficient in causal modeling (e.g., graphical mo

In [218]:
data_df

,job_link,job_title,salary,job_loc,company,job_description,experience_req,programming_skills,Education
0,http://www.glassdoor.sg/partner/jobListing.htm...,Sr. Data Scientist,$60K - $103K,"Atlanta, GA",Rubicon,SENIOR DATA SCIENTIST- Temporary Assignment (R...,3,R Python SQL Power BI,Bachelors Masters
1,http://www.glassdoor.sg/partner/jobListing.htm...,Data Scientist,$74K - $123K,"Atlanta, GA",Purchasing Power,Data Scientist Location Atlanta - Midtown Dep...,NaN,R Python,Bachelors Masters
2,http://www.glassdoor.sg/partner/jobListing.htm...,Data Engineer,$57K - $111K,"Atlanta, GA",Sharecare Inc,Sharecare is the digital health company that h...,NaN,R Python SQL Agile,Bachelors
3,http://www.glassdoor.sg/partner/jobListing.htm...,Palantir Foundry Trainer/Data Analyst,$50K - $90K,"Atlanta, GA",Shine Systems,Short term work for 2 mid’s or 1 sr is needed ...,NaN,Python,
4,http://www.glassdoor.sg/partner/jobListing.htm...,Business Intelligence Analyst,$47K - $96K,"Atlanta, GA",Purchasing Power,Business Intelligence Analyst Location Atlant...,2+,R Python SQL Tableau Visualization,Bachelors Masters
...,...,...,...,...,...,...,...,...,...
5499,http://www.glassdoor.sg/partner/jobListing.htm...,Principal Data Scientist,$104K - $170K,"Chevy Chase, MD",GEICO,"Working out of our Chevy Chase, MD/Washington ...",NaN,,
5500,http://www.glassdoor.sg/partner/jobListing.htm...,Senior Data Scientist - Finance,$122K - $194K,"Arlington, VA","Surescripts, LLC",Our purpose is to serve the nation with the si...,NaN,,
5501,http://www.glassdoor.sg/partner/jobListing.htm...,Sr. Data Analyst (ArcSight) with Top Secret/SCI,NaN,"Washington, DC",Trewon Technologies,Trewon is a competitive firm providing enterpr...,NaN,,
5502,http://www.glassdoor.sg/partner/jobListing.htm...,Senior Big Data Analyst,$74K - $125K,"Chevy Chase, MD",GEICO,The Data Consumption team of Decision Sciences...,2,,


## Programming Skills

In [219]:
# Create a data frame with skills name and regex pattern to search with
skills = ['R','Python','Hadoop','SQL','Tableau','TensorFlow','Agile','Power BI','SSaS','Algorithm','Java','Visualization']
# skills = ['Python']
# skill_patt = ['(^|\W)python($|\W)']
skill_patt = ['(^|\W)r($|\W)','(^|\W)python($|\W)','(^|\W)hadoop($|\W)','(^|\W)sql($|\W)','(^|\W)tableau($|\W)',
              "(^|\W)tensorflow($|\W)","(^|\W)agile($|\W)","(^|\W)powerbi($|\W)",
             "(^|\W)ssas($|\W)","(^|\W)algorithm($|\W)",'(^|\W)java($|\W)','(^|\W)visualization($|\W)']
# skill_patt = ['\WR\W+\s*','(?i)\WPython\W','(?i)\WHadoop\W?','(?i)SQL\w*','(?i)\WTableau\W?',
#               "(?i)\WTensorFlow\W?","(?i)\WAgile\W?","(?i)\WPower\s?BI\W?",
#              "(?i)\WSSAS\W?","(?i)\WAlgorithms?\W?",'(?i)Java\w*','(?i)\WVisualization\W?']

skill_df =pd.DataFrame(
    {"skill": skills,
     "regex_pattern":skill_patt})

In [220]:
# Iterate through the list of skill using the search function created
# data_df['programming_skills'] = np.nan
data_df['programming_skills'] = ''
for text_id in range(len(text)):
        job_desc = text[text_id].lower().replace("\n", " ")
        skills = []
        for skill_id in range(len(skill_df)):
            skill_pattern = skill_df['regex_pattern'][skill_id]
            skill_name = skill_df['skill'][skill_id]
            matcher = re.search(skill_pattern, job_desc)
            if matcher is not None:
                skills.append(skill_name)
        data_df['programming_skills'][text_id] = ' '. join(skills)

In [221]:
data_df

,job_link,job_title,salary,job_loc,company,job_description,experience_req,programming_skills,Education
0,http://www.glassdoor.sg/partner/jobListing.htm...,Sr. Data Scientist,$60K - $103K,"Atlanta, GA",Rubicon,SENIOR DATA SCIENTIST- Temporary Assignment (R...,3,R Python SQL Power BI,Bachelors Masters
1,http://www.glassdoor.sg/partner/jobListing.htm...,Data Scientist,$74K - $123K,"Atlanta, GA",Purchasing Power,Data Scientist Location Atlanta - Midtown Dep...,NaN,R Python,Bachelors Masters
2,http://www.glassdoor.sg/partner/jobListing.htm...,Data Engineer,$57K - $111K,"Atlanta, GA",Sharecare Inc,Sharecare is the digital health company that h...,NaN,R Python SQL Agile,Bachelors
3,http://www.glassdoor.sg/partner/jobListing.htm...,Palantir Foundry Trainer/Data Analyst,$50K - $90K,"Atlanta, GA",Shine Systems,Short term work for 2 mid’s or 1 sr is needed ...,NaN,Python,
4,http://www.glassdoor.sg/partner/jobListing.htm...,Business Intelligence Analyst,$47K - $96K,"Atlanta, GA",Purchasing Power,Business Intelligence Analyst Location Atlant...,2+,R Python SQL Tableau Visualization,Bachelors Masters
...,...,...,...,...,...,...,...,...,...
5499,http://www.glassdoor.sg/partner/jobListing.htm...,Principal Data Scientist,$104K - $170K,"Chevy Chase, MD",GEICO,"Working out of our Chevy Chase, MD/Washington ...",NaN,R Python Agile,
5500,http://www.glassdoor.sg/partner/jobListing.htm...,Senior Data Scientist - Finance,$122K - $194K,"Arlington, VA","Surescripts, LLC",Our purpose is to serve the nation with the si...,NaN,R Python SQL Tableau Visualization,
5501,http://www.glassdoor.sg/partner/jobListing.htm...,Sr. Data Analyst (ArcSight) with Top Secret/SCI,NaN,"Washington, DC",Trewon Technologies,Trewon is a competitive firm providing enterpr...,NaN,,
5502,http://www.glassdoor.sg/partner/jobListing.htm...,Senior Big Data Analyst,$74K - $125K,"Chevy Chase, MD",GEICO,The Data Consumption team of Decision Sciences...,2,Hadoop SQL Agile,


## Education

In [222]:
edu = ["Bachelors", "Bachelors" ,"Masters", "Masters", "PhD", "PhD"]
edu_patt = ["(?i)\WBachelors?'?s?\W", '(?i)\WB.?S\W', "(?i)\WMasters?'?s?\W", '(?i)\WM.?S\W', '(?i)\WPh.?D\W', '(?i)\WDoctorate\W']
# edu = ["Bachelors", "Masters", "PhD"]
# edu_patt = ['(?i)\WB.?S\W', "(?i)\WMasters?'?s?\W", '(?i)\WDoctorate\W']
edu_df =pd.DataFrame(
    {"education": edu,
     "regex_pattern":edu_patt})

In [224]:
# Iterate through the list of skill using the search function created
# data_df['programming_skills'] = np.nan
data_df['Education'] = ''
for text_id in range(len(text)):
        job_desc = text[text_id].lower().replace("\n", " ")
            #job_desc = text[text_id].lower().replace("\n", " ")
        edus = []
        for edu_id in range(len(edu_df)):
            edu_pattern = edu_df['regex_pattern'][edu_id]
            edu_name = edu_df['education'][edu_id]
            matcher = re.search(edu_pattern, job_desc)
            if matcher is not None:
                edus.append(edu_name)
        data_df['Education'][text_id] = ' '. join(edus)


In [170]:
type(text[1])

str

In [225]:
data_df

,job_link,job_title,salary,job_loc,company,job_description,experience_req,programming_skills,Education
0,http://www.glassdoor.sg/partner/jobListing.htm...,Sr. Data Scientist,$60K - $103K,"Atlanta, GA",Rubicon,SENIOR DATA SCIENTIST- Temporary Assignment (R...,3,R Python SQL Power BI,Bachelors Masters
1,http://www.glassdoor.sg/partner/jobListing.htm...,Data Scientist,$74K - $123K,"Atlanta, GA",Purchasing Power,Data Scientist Location Atlanta - Midtown Dep...,NaN,R Python,Bachelors Masters
2,http://www.glassdoor.sg/partner/jobListing.htm...,Data Engineer,$57K - $111K,"Atlanta, GA",Sharecare Inc,Sharecare is the digital health company that h...,NaN,R Python SQL Agile,Bachelors
3,http://www.glassdoor.sg/partner/jobListing.htm...,Palantir Foundry Trainer/Data Analyst,$50K - $90K,"Atlanta, GA",Shine Systems,Short term work for 2 mid’s or 1 sr is needed ...,NaN,Python,
4,http://www.glassdoor.sg/partner/jobListing.htm...,Business Intelligence Analyst,$47K - $96K,"Atlanta, GA",Purchasing Power,Business Intelligence Analyst Location Atlant...,2+,R Python SQL Tableau Visualization,Bachelors Masters
...,...,...,...,...,...,...,...,...,...
5499,http://www.glassdoor.sg/partner/jobListing.htm...,Principal Data Scientist,$104K - $170K,"Chevy Chase, MD",GEICO,"Working out of our Chevy Chase, MD/Washington ...",NaN,R Python Agile,
5500,http://www.glassdoor.sg/partner/jobListing.htm...,Senior Data Scientist - Finance,$122K - $194K,"Arlington, VA","Surescripts, LLC",Our purpose is to serve the nation with the si...,NaN,R Python SQL Tableau Visualization,Bachelors
5501,http://www.glassdoor.sg/partner/jobListing.htm...,Sr. Data Analyst (ArcSight) with Top Secret/SCI,NaN,"Washington, DC",Trewon Technologies,Trewon is a competitive firm providing enterpr...,NaN,,Bachelors
5502,http://www.glassdoor.sg/partner/jobListing.htm...,Senior Big Data Analyst,$74K - $125K,"Chevy Chase, MD",GEICO,The Data Consumption team of Decision Sciences...,2,Hadoop SQL Agile,Bachelors Masters


In [227]:
data_df['job_description'][10]

"Trellis Rx is a technology-enabled service provider that partners with health systems to fund, build, and operate world-class specialty pharmacies on their behalf, with their brand. Trellis enables health systems to deliver enhanced experiences and outcomes by helping them offer concierge level clinical and administrative support to patients throughout their course of therapy. We believe pharmacists and pharmacy technicians are critical additions to the care team and that an exceptional pharmacy team makes superior results possible. The Senior Healthcare Data Analyst provides analytical and consultative support to our prospective customers and to our commercial organization to accurately model and design recommended specialty pharmacy services. The primary focus of the position is to help to develop financial and operational plans for the creation of a new or accelerated specialty pharmacy service lines within leading health systems across the country. Working alongside the Vice Presi

In [228]:
data_df['experience_req'][10]

nan

In [226]:
data_df.to_csv("final_data.csv")

In [ ]:
from sklearn.model_selection import train_test_split
